In [1]:
import tensorflow.compat.v1 as tf
import lucid_kietzmannlab.modelzoo.vision_models as models
import matplotlib.pyplot as plt
from tqdm import tqdm
import lucid_kietzmannlab.modelzoo.vision_models as models
import lucid_kietzmannlab.optvis.objectives as objectives
import lucid_kietzmannlab.optvis.render as render
from IPython.display import clear_output
from ipywidgets import interact, Dropdown, IntSlider
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
model_checkpoint_dir = "/Users/vkapoor/Downloads/training_seed_05"
model_checkpoint = "model.ckpt_epoch89"
model, graph, logits, activations, weights, layer_shape_dict = models.load_ecoset_model_seeds(model_checkpoint_dir, model_checkpoint)


INFO:tensorflow:Restoring parameters from /Users/vkapoor/Downloads/training_seed_05/model.ckpt_epoch89


2024-06-08 08:31:33.256759: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


Model loaded from: /Users/vkapoor/Downloads/training_seed_05


/Users/vkapoor/miniconda3/envs/lucidenv/lib/python3.12/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [3]:
for layer, shape in layer_shape_dict.items():
    print(f'Layer: {layer}, Shape: {shape}')

Layer: alexnet_v2/conv1/Conv2D, Shape: (?, 54, 54, 64)
Layer: alexnet_v2/conv2/Conv2D, Shape: (?, 26, 26, 192)
Layer: alexnet_v2/conv3/Conv2D, Shape: (?, 12, 12, 384)
Layer: alexnet_v2/conv4/Conv2D, Shape: (?, 12, 12, 384)
Layer: alexnet_v2/conv5/Conv2D, Shape: (?, 12, 12, 256)
Layer: alexnet_v2/fc6/Conv2D, Shape: (?, 1, 1, 4096)
Layer: alexnet_v2/fc7/Conv2D, Shape: (?, 1, 1, 4096)
Layer: alexnet_v2/fc8/Conv2D, Shape: (?, 1, 1, 565)


In [4]:
C = lambda neuron: objectives.channel(*neuron)    
def visualize(layer_name, channel):
    # Check if the layer exists in the shape dictionary
    if layer_name in layer_shape_dict:
        # Check if the selected channel is within bounds
        print(layer_shape_dict[layer_name])
        max_channel = layer_shape_dict[layer_name][-1] - 1
        if 0 <= channel <= max_channel:
            #clear_output(wait=True)
            # Render visualization for the selected layer and channel
            #try:
            _ = render.render_vis(model, C((layer_name, channel)), scope='alexnet_v2')
            #except Exception:
            #    print('No gradients for this layer')   

def visualize_all():
    # Check if the layer exists in the shape dictionary
    layer_name = current_dropdown_value({'new': layer_dropdown.value})
    print(layer_name)
    if layer_name in layer_shape_dict:
            # Check if the selected channel is within bounds
            try:
               image_channel = {} 
               for channel in tqdm(range(channel_slider.max)):    
                  images = render.render_vis(model, C((layer_name, channel)), verbose = False)
                  image_channel[channel] = images 
            except Exception:
                print('No gradients for this layer')    
    return image_channel   


# Create dropdown menu for layer selection
layer_dropdown = Dropdown(options=list(layer_shape_dict.keys()), description='Layer:')

# Create slider for channel selection
channel_slider = IntSlider(min=0, max=0, description='Channel:')
        
        
def update_channel_slider(change):
    layer_name = change.new
    if layer_name in layer_shape_dict:
        print(layer_name)
        print(layer_shape_dict[layer_name])
        max_channel = layer_shape_dict[layer_name][-1] - 1
        channel_slider.max = max_channel
        
        
def current_slider_value(*args):
    return channel_slider.value


def current_dropdown_value(change):
    return change['new']


channel_slider.observe(current_slider_value, names='value')
layer_dropdown.observe(current_dropdown_value, names='value')
          

In [5]:
layer_dropdown.observe(update_channel_slider, names='value')

# Create an interactive visualization
interact(visualize, layer_name=layer_dropdown, channel=channel_slider)     


interactive(children=(Dropdown(description='Layer:', options=('alexnet_v2/conv1/Conv2D', 'alexnet_v2/conv2/Con…

<function __main__.visualize(layer_name, channel)>

In [6]:
lname = 'alexnet_v2/conv1/Conv2D'
visualize(lname, 1)

(?, 54, 54, 64)


2024-06-08 08:31:45.835540: W tensorflow/core/common_runtime/graph_constructor.cc:1595] Importing a graph with a lower producer version 38 into an existing graph with producer version 1766. Shape inference will have run different parts of the graph with different producer versions.


ValueError: Can not squeeze dim[3], expected a dimension of 1, got 3 for '{{node alexnet_v2/tower_0/alexnet_v2/fc8/squeezed}} = Squeeze[T=DT_FLOAT, squeeze_dims=[2, 3]](alexnet_v2/tower_0/alexnet_v2/fc8/BiasAdd)' with input shapes: [32,565,?,3].